In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [2]:
# Load CIFAR-10 dataset and preprocess
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = tf.image.resize(x_train / 255.0, (64, 64)), tf.image.resize(x_test / 255.0, (64, 64))

In [3]:
# Load pre-trained VGG16 model, freeze base layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
for layer in base_model.layers: layer.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 19s 0us/step


In [4]:
# Add custom classifier
x = Flatten()(base_model.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(10, activation='softmax')(x)

In [5]:
# Create and compile model
model = Model(inputs=base_model.input, outputs=x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train and evaluate model
model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test), batch_size=64)
test_acc = model.evaluate(x_test, y_test, verbose=0)[1]


Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 425s 541ms/step - accuracy: 0.3801 - loss: 1.7561 - val_accuracy: 0.6180 - val_loss: 1.1420
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 445s 570ms/step - accuracy: 0.5410 - loss: 1.3160 - val_accuracy: 0.6343 - val_loss: 1.0867
Epoch 3/5
 28/782 ━━━━━━━━━━━━━━━━━━━━ 5:52 468ms/step - accuracy: 0.5653 - loss: 1.2770

In [ ]:
# Predict and visualize results
predictions = model.predict(x_test)
plt.imshow(x_test[10])
plt.title(f"Pred: {predictions[10].argmax()}, Actual: {y_test[10][0]}")
plt.show()


In [ ]:
print(f'Test accuracy: {test_acc * 100:.2f}%')
